In [1]:
import collections
import re
from d2l import torch as d2l

`read_time_machine` 将小说数据集加载到文本行的列表中，且将所有大写都换成小写字符，标点符号换成空格，最后返回一个列表

In [16]:
def read_time_machine():
    with open('../data/timemachineBook/timemachine.txt', 'r') as f:
        lines = f.readlines()
    """ 把不是字母的东西都变成空格，其实就是把标点符号变成空格 """
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines] # strip() 去除行符

In [15]:
lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


`tokenize` 函数将 `lines` 中每一段文本序列拆分成词元（token），词元可以是一个单词或者一个字符串组成

In [17]:
def tokenize(lines, token='word'):
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('wrong token:' + token)

In [42]:
tokens_word = tokenize(lines, token='word')
tokens_word[0]

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']

### 词表（字典）

`Vocab` 类将词元列表变为带有下标的字典，形式：{'a': 0, 'adan': 1}，最终结果在 `self.token_to_idx` 这个参数中，对词元进行频率统计得到结果称为语料（corpus）

In [44]:
class Vocab:
    """ reserved_tokens 说明句子开始和结束 """
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens) # 得到词元出现次数
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True) # 按出现频率排序，由大到小 
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq: # 词元出现次数太少就不做统计
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token) # 将当前词元添加到列表中，这个 idx_to_token 列表是为了拿到下标做准备，形式：['unk', 'the']
                self.token_to_idx[token] = len(self.idx_to_token) - 1 # 形式：{'unk': 0, 'the': 1}，这里 len(self.idx_to_token) - 1 是下标，减去 1 是因为之前有一个 'unk'
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return [self.idx_to_token[index] for index in indices]
    
    def unk(self):
        return 0
    
    def token_freq(self):
        return self._token_freq

`count_corpus` 计算一个词元列表中所有元素的出现频率，且返回一个字典，词元作为 key，频率作为 value

In [29]:
def count_corpus(tokens):
    if len(tokens) == 0 or isinstance(tokens[0], list): # isinstance() 函数是判断一个对象是否为传入的类型或者子类
        tokens = [token for line in tokens for token in line] # 两个循环先前面再后面
    return collections.Counter(tokens)

In [66]:
vocab = Vocab(tokens_word)
# list(vocab.token_to_idx.items())[:5]

22

In [56]:
for i in [0, 10]:
    print('文本', tokens_word[i])
    print('索引', vocab[tokens_word[i]])

文本 ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引 [1, 19, 50, 40, 2183, 2184, 400]
文本 ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引 [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


### 功能打包

`load_corpus_time_machine` 函数将以上功能打包，返回词元索引列表 `corpus` 和 语料库的词表（索引字典） `vovab`

In [69]:
def load_corpus_time_machine(max_token=-1):
    lines = read_time_machine()
    tokens = tokenize(lines, token='char')
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line]
    if max_token > 0:
        corpus = corpus[:max_token]
    return corpus, vocab